In [3]:
import re
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from gensim.models import Word2Vec, KeyedVectors

In [2]:
col_names = ['id', 'date', 'username', 'text', 'type', 'replies', 'favs', 'messages', 'followers', 'friends', 'lists']
pos_db = pd.read_csv('positive.csv', delimiter=';', names=col_names).sample(1000)['username']
neg_db = pd.read_csv('negative.csv', delimiter=';', names=col_names).sample(1000)['username']

In [14]:
texts = pd.concat([pos_db, neg_db], ignore_index=True).values

In [16]:
np.random.shuffle(texts)

In [17]:
texts

array(['снова в кино.\nфильм оказался уж точно не моим.\nхорошо что есть книга)',
       '@_8763731544813 о! пугач! значит я не молодежь? (((',
       'братан в парочке♥\nзаюша Веруша:* http://t.co/qwTgHq6R3I', ...,
       '"@taniastern: у дженнифер эннистон оказывается было 2 выкидыша, а у кортни кокс целых 8" ОБОЖЕ!!! Ужасссс(',
       'Вот откуда любовь к совам....\nИли к Франции))) http://t.co/qW1Flq3cUj',
       'RT @domanep: Классно будет,если создатели Subway Surf придумали новую тему в Хогвартсе:пробегать сквозь стены,собирать волшебные палочки...)'], dtype=object)

## Preprocessing

In [34]:
m = Mystem()
def lemmatize(sent):
    to_lemmatize = []
    words = re.findall('\w+', sent)
    for word in words:
        if not word.startswith('@') or not word.startswith('http'):
            if word != 'RT':
                if word.isalpha():
                    if re.findall('[a-zA-Z]', word)==[]:
                        to_lemmatize.append(word)
    lemmas = []
    #processed_sent = ' '.join(to_lemmatize)
    for l in to_lemmatize:
        try:
            lemma = m.analyze(l)[0]['analysis'][0]['lex'] + '_' + m.analyze(l)[0]['analysis'][0]['gr'].split(',')[0]
            if '=' in lemma:
                lemma = lemma[:lemma.find('=')]
            lemmas.append(lemma)
        except IndexError:
            pass
    processed_sent = ' '.join(lemmas)
    return lemmas, processed_sent

Собираем список лемм (расклеиваем склеившиеся строки)

In [5]:
#ненавидеть_Vэто_SPRO
ls = open('lemmas.txt', 'r', encoding='utf8').readlines()
patt = re.compile('[A-Z][а-я]')
lemmas = set()
for l in ls:
    if re.findall(patt, l) != []:
        for i,el in enumerate(l.split('_')[1]):
            if el.lower() == el:
                idx = i + len(l.split('_')[0]) + 1
                break
        lemmas.add(l[:idx])
        lemmas.add(l[idx:][:-1])
    else:
        lemmas.add(l[:-1])

In [6]:
twits = open('twits.txt', 'r', encoding='utf8').read().split('\n*****\n')

## Word2Vec

In [36]:
from gensim.models import Word2Vec, KeyedVectors

In [37]:
model = KeyedVectors.load_word2vec_format('ruscorpora_mystem_cbow_300_2_2015.bin.gz', binary=True)

In [118]:
word_vecs = {}
for lemma in list(lemmas):
    try:
        word_vecs[lemma] = model[lemma]
    except KeyError:
        pass
with open('dict.txt', 'w', encoding='utf8') as d:
    for word in word_vecs:
        to_write = word + ' : ' + str(word_vecs[word])
        d.write(to_write)
        d.write('\n')

In [104]:
len(word_vecs)

1622

## Doc2Vec

In [170]:
doc_vecs = []
for doc in twits:
    matr = []
    for word in doc.split(' '):
        if word in word_vecs.keys():
            matr.append(list(word_vecs[word]))
    doc_vec = []
    for i in range(0,300):
        try:
            doc_vec.append(max([line[i] for line in matr]))
        except ValueError:
            continue
    doc_vecs.append(doc_vec)

In [172]:
len(doc_vecs)

501

In [173]:
with open('doc_vecs.txt', 'w', encoding='utf8') as dv:
    for vec in doc_vecs:
        dv.write(str(vec))
        dv.write('\n')

## Fast text

In [1]:
from gensim.models import FastText

c:\users\aaleshin\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
ftexts = []
for twit in twits:
    ftexts.append([el.split('_')[0] for el in twit.split(' ')])

In [14]:
model_ft = FastText(ftexts, size=200, min_count=1, iter=40, word_ngrams=3)

In [18]:
ft_dict = {}
for lemma in lemmas:
    ft_dict[lemma.split('_')[0]] = model_ft[lemma.split('_')[0]]

c:\users\aaleshin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
len(ft_dict)

1907